## Коллаборативная фильтрация user2user

Используем датасет MovieLens с оценками пользователями фильмов

In [ ]:
!pip install "numpy<2"
import numpy as np
!pip install scikit-surprise

from math import sqrt
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_builtin('ml-100k', reader)

trainset, valset = train_test_split(data, test_size=0.2)

all_users = set([trainset.to_raw_uid(user) for (user, item, rating) in trainset.all_ratings()]).union(set([user for (user, item, rating) in valset]))
all_items = set([trainset.to_raw_iid(item) for (user, item, rating) in trainset.all_ratings()]).union(set([item for (user, item, rating) in valset]))

user_map = {user: i for i, user in enumerate(all_users)}
item_map = {item: i for i, item in enumerate(all_items)}

n_users = len(user_map)
n_items = len(item_map)

trainset_ratings=np.zeros(shape=(n_users, n_items))
valset_ratings = []
valset_matrix = np.zeros(shape=(n_users, n_items))
for (user, item, rating) in trainset.all_ratings():
  trainset_ratings[user_map[trainset.to_raw_uid(user)]][item_map[trainset.to_raw_iid(item)]] = rating
for (user, item, rating) in valset:
  valset_ratings.append((user_map[user], item_map[item], rating))
  valset_matrix[user_map[user]][item_map[item]] = rating

trainset_ratings = pd.DataFrame(trainset_ratings)

In [ ]:
print("Пользователей:", n_users)
print("Items (фильмов):", n_items)

Пользователей: 943
Items (фильмов): 1682


In [ ]:
trainset_ratings

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,0.0,3.0,0.0,3.0,0.0,1.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,5.0,5.0,3.0,0.0,0.0
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,5.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
939,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


# Функции схожести

Простое произведение (sim_dot)

$\text{sim}_{\text{dot}}(\mathbf{u}, \mathbf{v}) = \mathbf{u} \cdot \mathbf{v} = \sum_{i} u_i v_i$

In [ ]:
def sim_dot(left, right) -> float:

    sim = np.dot(left, right)
    return sim

Коэффициент Жаккара для рейтингов (коэффициент Танимото) (sim_jacc)

$ \text{sim}_{\text{jacc}}(\mathbf{u}, \mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{||\mathbf{u}||^2_2 + ||\mathbf{v}||^2_2 - \mathbf{u} \cdot \mathbf{v}} $

In [ ]:
def sim_jacc(left, right) -> float:

    dot = np.dot(left, right)
    denominator = np.dot(left,left) + np.dot(right,right) - dot
    if denominator == 0:
        return 0
    sim = dot/denominator
    return sim

Корреляция Пирсона (sim_pearson)

$ \text{sim}_{\text{pearson}}(\mathbf{u}, \mathbf{v}) = \frac{\sum_{i \in I}(u_i - \bar{u}_I)(v_i - \bar{v}_I)}{\sqrt{\sum_{i \in I}(u_i - \bar{u}_I)^2} \sqrt{\sum_{i \in I}(v_i - \bar{v}_I)^2}} $

In [ ]:
def sim_pearson(left, right) -> float:

    common_mask = (left != 0) & (right != 0)

    if not np.any(common_mask):
        return 0.0

    left_common = left[common_mask]
    right_common = right[common_mask]

    left_mean = np.mean(left_common)
    right_mean = np.mean(right_common)

    left_centered = left_common - left_mean
    right_centered = right_common - right_mean

    numerator = np.sum(left_centered * right_centered)
    denominator = np.sqrt(np.sum(left_centered**2) * np.sum(right_centered**2))

    if denominator == 0:
        return 0.0

    return numerator / denominator

Корреляция Пирсона с поправкой (sim_pearson_decreasing)

$ \text{sim}_{\text{pearson_dec}}(\mathbf{u}, \mathbf{v}) = \min\left(\frac{|I|}{50}, 1\right) \cdot \text{sim}_{\text{pearson}}(\mathbf{u}, \mathbf{v}) $

In [ ]:
def sim_pearson_decreasing(left, right) -> float:

    common_mask = (left != 0) & (right != 0)

    if not np.any(common_mask):
        return 0.0

    left_common = left[common_mask]
    right_common = right[common_mask]

    left_mean = np.mean(left_common)
    right_mean = np.mean(right_common)

    left_centered = left_common - left_mean
    right_centered = right_common - right_mean

    numerator = np.sum(left_centered * right_centered)
    denominator = np.sqrt(np.sum(left_centered**2) * np.sum(right_centered**2))

    if denominator == 0:
        return 0.0

    sim = numerator / denominator
    numintersect = np.count_nonzero(common_mask)
    return min(numintersect / 50, 1) * sim

## Алгоритм коллаборативной фильтрации user2user

**Принцип работы**

1.  Вычисление матрицы схожести: Для каждой пары пользователей рассчитывается мера схожести их векторов оценок с помощью функции sim_fn.
2.  Предсказание оценок: Для целевого пользователя и каждого непросмотренного им товара:
    *   Находим всех пользователей, которые оценили этот товар
    *   Взвешиваем их оценки на степень схожести с целевым пользователем
    *   Нормируем на сумму весов (схожестей) для получения итоговой прогнозируемой оценки
3.  Рекомендация: Выбираются товары с наивысшими прогнозируемыми оценками.

**Основные методы:**

1.  calc_sim_matrix(feedbacks):
    *   Вычисляет и сохраняет симметричную матрицу попарных схожестей всех пользователей
    *   *Вход*: feedbacks — 2D NumPy array, где строка = пользователь, столбец = товар

2.  recommend(user, n):
    *   Возвращает список из n рекомендованных товаров для пользователя user
    *   Рекомендуются только товары, которые пользователь еще не видел (оценка = 0)

**Формула предсказания**

В зависимости от флага mean_correct используется одна из двух формул:

Без коррекции на среднее (mean_correct = False):
$$
\hat{r}_{u, i} = \frac{\sum_{v \in U_i} \text{sim}(u, v) \cdot r_{v, i}}{\sum_{v \in U_i} \text{sim}(u, v)}
$$

С коррекцией на среднее (mean_correct = True):
$$
\hat{r}_{u, i} = \bar{r}_u + \frac{\sum_{v \in U_i} \text{sim}(u, v) \cdot (r_{v, i} - \bar{r}_v)}{\sum_{v \in U_i} \text{sim}(u, v)}
$$

Где:
*   $\hat{r}_{u, i}$ — прогнозируемая оценка пользователя $u$ для товара $i$
*   $U_i$ — множество пользователей, оценивших товар $i$
*   $\text{sim}(u, v)$ — схожесть пользователей $u$ и $v$
*   $r_{v, i}$ — известная оценка пользователя $v$ для товара $i$
*   $\bar{r}_u$, $\bar{r}_v$ — средние оценки пользователей $u$ и $v$ (по поставленным ими оценкам)

Коррекция на среднее позволяет учесть личные предпочтения пользователей (например, один завышает оценки, другой — занижает).


In [ ]:
class UserBasedCf:
    def __init__(self, sim_fn, mean_correct: bool = False):
        self.sim_fn = sim_fn
        self.mean_correct = mean_correct

    def calc_sim_matrix(self, feedbacks):
        self.feedbacks = np.array(feedbacks)

        num_users = feedbacks.shape[0]
        self.sim_matrix = np.zeros((num_users, num_users))

        for i in range(num_users):
            for j in range(i + 1, num_users):
                if i != j:
                    similarity = self.sim_fn(feedbacks[i], feedbacks[j])
                    self.sim_matrix[i, j] = similarity
                    self.sim_matrix[j, i] = similarity

    def recommend(self, user: int, n: int):
        user_ratings = self.feedbacks[user]
        unseen_items = np.where(user_ratings == 0)[0]

        if len(unseen_items) == 0:
            return []

        predicted_ratings = np.zeros(len(unseen_items))

        for idx, item in enumerate(unseen_items):
            sim_sum = 0
            weighted_ratings_sum = 0

            for other_user in range(self.feedbacks.shape[0]):
                if other_user != user and self.feedbacks[other_user, item] > 0:
                    similarity = self.sim_matrix[user, other_user]
                    sim_sum += similarity

                    if self.mean_correct:
                        other_user_mean = np.mean(self.feedbacks[other_user][self.feedbacks[other_user] > 0])
                        weighted_ratings_sum += similarity * (self.feedbacks[other_user, item] - other_user_mean)
                    else:
                        weighted_ratings_sum += similarity * self.feedbacks[other_user, item]

            if sim_sum > 0:
                if self.mean_correct:
                    user_mean = np.mean(self.feedbacks[user][self.feedbacks[user] > 0])
                    predicted_ratings[idx] = user_mean + (weighted_ratings_sum / sim_sum)
                else:
                    predicted_ratings[idx] = weighted_ratings_sum / sim_sum

        recommended_indices = np.argsort(predicted_ratings)[-n:][::-1]
        return unseen_items[recommended_indices].tolist()

In [ ]:
class UserBasedCf:
    def __init__(self, sim_fn, mean_correct: bool = False):
        self.sim_fn = sim_fn
        self.mean_correct = mean_correct
        self.user_means = None

    def calc_sim_matrix(self, feedbacks):
        # Гарантируем, что feedbacks - это двумерный numpy array
        self.feedbacks = np.array(feedbacks)
        if self.feedbacks.ndim == 1:
            self.feedbacks = self.feedbacks.reshape(1, -1)

        num_users, num_items = self.feedbacks.shape

        # Предварительно вычисляем средние для пользователей
        if self.mean_correct:
            self.user_means = np.zeros(num_users)
            for i in range(num_users):
                user_ratings = self.feedbacks[i]
                rated_items = user_ratings > 0
                if np.any(rated_items):
                    self.user_means[i] = np.mean(user_ratings[rated_items])

        # Вычисляем матрицу схожести
        self.sim_matrix = np.zeros((num_users, num_users))

        for i in range(num_users):
            for j in range(i + 1, num_users):  # Используем симметричность
                sim = self.sim_fn(self.feedbacks[i], self.feedbacks[j])
                self.sim_matrix[i, j] = sim
                self.sim_matrix[j, i] = sim

    def recommend(self, user: int, n: int):
        if not hasattr(self, 'feedbacks'):
            raise ValueError("Сначала вызовите calc_sim_matrix()")

        user_ratings = self.feedbacks[user]
        unseen_items = np.where(user_ratings == 0)[0]

        if len(unseen_items) == 0:
            return []

        # Получаем схожести с текущим пользователем (один раз)
        user_sims = self.sim_matrix[user, :]

        if self.mean_correct:
            # Предварительные вычисления для mean correction
            user_mean = self.user_means[user]
            deviations = self.feedbacks - self.user_means[:, np.newaxis]

            # Выбираем только нужные столбцы (unseen_items)
            deviations_subset = deviations[:, unseen_items]
            mask_subset = (deviations_subset != 0)  # Маска ненулевых отклонений

            # Векторизованные вычисления
            weighted_sum = (user_sims[:, np.newaxis] * deviations_subset * mask_subset).sum(axis=0)
            sim_sum = (np.abs(user_sims[:, np.newaxis]) * mask_subset).sum(axis=0)

        else:
            # Выбираем только нужные столбцы (unseen_items)
            ratings_subset = self.feedbacks[:, unseen_items]
            mask_subset = (ratings_subset > 0)

            # Векторизованные вычисления
            weighted_sum = (user_sims[:, np.newaxis] * ratings_subset * mask_subset).sum(axis=0)
            sim_sum = (np.abs(user_sims[:, np.newaxis]) * mask_subset).sum(axis=0)

        # Быстрое вычисление предсказанных рейтингов
        valid_mask = sim_sum > 0
        predicted_ratings = np.zeros(len(unseen_items))

        if np.any(valid_mask):
            if self.mean_correct:
                predicted_ratings[valid_mask] = user_mean + weighted_sum[valid_mask] / sim_sum[valid_mask]
            else:
                predicted_ratings[valid_mask] = weighted_sum[valid_mask] / sim_sum[valid_mask]

            # Используем argpartition для частичной сортировки (быстрее чем argsort)
            if len(unseen_items) > n:
                # Находим индексы n наибольших элементов
                top_n_indices = np.argpartition(-predicted_ratings, n)[:n]
                # Сортируем только top n элементов
                sorted_top_indices = top_n_indices[np.argsort(-predicted_ratings[top_n_indices])]
                return unseen_items[sorted_top_indices].tolist()
            else:
                # Если элементов меньше n, просто сортируем все
                sorted_indices = np.argsort(-predicted_ratings)
                return unseen_items[sorted_indices].tolist()

        return []

# Валидация моделей

In [ ]:
model1 = UserBasedCf(sim_dot, mean_correct=True)
model1.calc_sim_matrix(trainset_ratings)
print("Model 1 - ok")
model2 = UserBasedCf(sim_dot, mean_correct=False)
model2.calc_sim_matrix(trainset_ratings)
print("Model 2 - ok")
model3 = UserBasedCf(sim_jacc, mean_correct=True)
model3.calc_sim_matrix(trainset_ratings)
print("Model 3 - ok")
model4 = UserBasedCf(sim_jacc, mean_correct=False)
model4.calc_sim_matrix(trainset_ratings)
print("Model 4 - ok")
model5 = UserBasedCf(sim_pearson, mean_correct=True)
model5.calc_sim_matrix(trainset_ratings)
print("Model 5 - ok")
model6 = UserBasedCf(sim_pearson, mean_correct=False)
model6.calc_sim_matrix(trainset_ratings)
print("Model 6 - ok")
model7 = UserBasedCf(sim_pearson_decreasing, mean_correct=True)
model7.calc_sim_matrix(trainset_ratings)
print("Model 7 - ok")
model8 = UserBasedCf(sim_pearson_decreasing, mean_correct=False)
model8.calc_sim_matrix(trainset_ratings)
print("Model 8 - ok")

Model 1 - ok
Model 2 - ok
Model 3 - ok
Model 4 - ok
Model 5 - ok
Model 6 - ok
Model 7 - ok
Model 8 - ok


In [ ]:
models=[model1, model2, model3, model4, model5, model6, model7, model8]
rec_matrices=[]
for i in range(1, 9):
  print("model",i)
  model = models[i-1]
  rec_matrix=[]
  for user in range(n_users):
    if user%200==0: print("user",user)
    rec_matrix.append(model.recommend(user, n_items))
  rec_matrices.append(rec_matrix)

model 1
user 0
user 200
user 400
user 600
user 800
model 2
user 0
user 200
user 400
user 600
user 800
model 3
user 0
user 200
user 400
user 600
user 800
model 4
user 0
user 200
user 400
user 600
user 800
model 5
user 0
user 200
user 400
user 600
user 800
model 6
user 0
user 200
user 400
user 600
user 800
model 7
user 0
user 200
user 400
user 600
user 800
model 8
user 0
user 200
user 400
user 600
user 800


Будем использовать следующие метрики для оцени качества коллаборативной фильтрации

* MAP@15: Среднее значение AP@k по всем пользователям, учитывающая позицию рекомендаций, так как использует AP@k.
* NDCG@15: Нормализованная версия DCG, которая позволяет сравнивать результаты для разных пользователей, учитывает ранжирование рекомендаций и их релевантность.
* AvgScore@15: Средняя оценка рекомендаций из топ-15 вывода метода, которые есть в валидационной выборке.

Посчитаем метрики и сравним алгоритмы

In [ ]:
def average_precision_at_k(recommended_items, relevant_items, k):
    if not relevant_items:
        return np.nan

    relevant_set = set(relevant_items)
    num_relevant = min(len(relevant_set), k)

    hit_count = 0.0
    precision_sum = 0.0

    for i, item in enumerate(recommended_items[:k], 1):
        if item in relevant_set:
            hit_count += 1
            precision_sum += hit_count / i

    return precision_sum / num_relevant

def ndcg_at_k(recommended_items, ratings, k):
    k = min(k, len(recommended_items))
    if k == 0: return np.nan
    dcg = np.sum([(2 ** (ratings[recommended_items[i - 1]]) - 1) / np.log2(i + 1) for i in range(1, k + 1)])
    idcg = np.sum([(2 ** 5 - 1) / np.log2(i + 1) for i in range(1, k + 1)])
    return dcg / idcg

def avg_score_at_k(recommended_items, ratings, k):
    recommended_items = recommended_items[:k]
    if not recommended_items: return np.nan
    relevant_scores = [ratings[item] for item in recommended_items]
    return np.mean(relevant_scores)

def get_relevant_items(triplets): #Создаем словарь для каждого пользователя - индексы фильмов с хорошими оценками
    relevant_items_dict = {}
    for user, item, rating in triplets:
        if rating >= 4: #Рейтинг от 4 считаем хорошим
            if user not in relevant_items_dict:
                relevant_items_dict[user] = []
            relevant_items_dict[user].append(item)
    return relevant_items_dict

In [ ]:
k = 10
for model_num in range(8):
  print("Model",model_num+1)
  recommended_items_arrays=rec_matrices[model_num] #Список рекомендаций по пользователям
  filtered_recommendations = []

  for user_id in range(n_users):
      user_recs = recommended_items_arrays[user_id]
      filtered_recs = []
      for item in user_recs:
          if valset_matrix[user_id, item] > 0:
              filtered_recs.append(item)

      filtered_recommendations.append(filtered_recs)
  relevant_items_dict = get_relevant_items(valset_ratings) #Список понравившихся из тестового набора

  apks = []
  for user in range(n_users):
      recommended_items = filtered_recommendations[user]
      k1 = min(k, len(recommended_items))
      relevant_items = relevant_items_dict.get(user, [])
      apk = average_precision_at_k(recommended_items, relevant_items, k1)
      apks.append(apk)
  map_k = np.nanmean(apks)
  print(f'MAP@{k}: {map_k}')

  ndcg_k = []
  for user in range(n_users):
      recommended_items = filtered_recommendations[user]
      ndcg = ndcg_at_k(recommended_items, valset_matrix[user], k)
      ndcg_k.append(ndcg)
  print(f'NDCG@{k}: {np.nanmean(ndcg_k)}')

  avg_score_k = []
  for user in range(n_users):
      recommended_items = filtered_recommendations[user]
      avg_score = avg_score_at_k(recommended_items, valset_matrix[user], k)
      avg_score_k.append(avg_score)
  print(f'AvgScore@{k}: {np.nanmean(avg_score_k)}')
  print()

Model 1
MAP@10: 0.6921721622123084
NDCG@10: 0.5411083021942678
AvgScore@10: 3.7631399389369635

Model 2
MAP@10: 0.7439872637259412
NDCG@10: 0.5676990826321768
AvgScore@10: 3.8235012566840414

Model 3
MAP@10: 0.6880545006750687
NDCG@10: 0.5397478002120204
AvgScore@10: 3.7613333502015753

Model 4
MAP@10: 0.7470083568854272
NDCG@10: 0.5697106412320443
AvgScore@10: 3.827539513857261

Model 5
MAP@10: 0.6868256191667488
NDCG@10: 0.5385330080927431
AvgScore@10: 3.760589460722298

Model 6
MAP@10: 0.6811771791848785
NDCG@10: 0.5322013313695341
AvgScore@10: 3.765052797597962

Model 7
MAP@10: 0.7028728678166114
NDCG@10: 0.5471134619038045
AvgScore@10: 3.775360980382234

Model 8
MAP@10: 0.7119700963718821
NDCG@10: 0.5504250561675178
AvgScore@10: 3.806072988883828

